In [2]:
import pandas as pd
from duckdb import query

In [3]:
pd.options.display.max_columns = 200
pd.options.display.max_rows = 500
pd.set_option('mode.chained_assignment', None)

## Join Tables

In [4]:
# these files are the output of Tyndale/code/01. Tyndale txt to pkl.py
# which takes the txt files as its input in Tyndale/data
# i'm going to join them all together and clean things up, add some new columns, etc.
# then make pkl files for django models
a = pd.read_pickle("../data/tagnt.pkl")
b = pd.read_pickle("../data/strongs.pkl")
c = pd.read_pickle("../data/tegmc.pkl")
d = pd.read_pickle("../data/bcvIndex.pkl")

# lexnalt data from biblehub
e = pd.read_pickle('../../pickles/lexnalt.pkl')

In [5]:
# see what's in each table, define fields to keep going forward
print('tagnt columns: ',a.columns)
print('strongs columns: ',b.columns)
print('tegmc columns: ',c.columns)
print('bcvIndex columns: ',d.columns)
tagnt_fields = ['bcv','greek','english', 'strongs_id', 'morphology_id',]
strongs_fields = ['strongs', 'lexicon', 'gloss', 'transliteration','definitionhtml',]
tegmc_fields = ['morphology','function', 'tense','voice','mood','person','case','gender','number'] 
bcvIndex_fields = ['bcv','book','bookLong']

tagnt columns:  Index(['reference', 'wordType', 'greek', 'english', 'strongs_id',
       'morphology_id', 'dictionaryForm', 'gloss', 'editions',
       'spellingVariants', 'meaningVariants', 'spanish', 'subMeaning',
       'superMeaning', 'conjoin', 'word', 'id', 'bcvIndex_id', 'bcv',
       'nestleAland'],
      dtype='object')
strongs columns:  Index(['strongs', 'frequency', 'gloss', 'lexicon', 'transliteration',
       'grammar', 'definitionhtml'],
      dtype='object')
tegmc columns:  Index(['morphology', 'function', 'tense', 'voice', 'mood', 'person', 'case',
       'gender', 'number', 'FORM', 'NAME_TYPE', 'EXTRA', 'ORIG_LANG',
       'ADJ_NUMB', 'INDECLINABLE', 'NAME_IN_ORIG_LANG'],
      dtype='object')
bcvIndex columns:  Index(['bcv', 'book', 'chapter', 'verse', 'bookLong'], dtype='object')


In [6]:
## not perfect, but forcing strongs ids to join 1-to-1 to strongs table
## some of the words had two or more strongs ids associated with them,
## so i'm keeping the first one in that case
a['strongsidlength'] = a.strongs_id.apply(lambda x: len(x))
print(a.groupby(['strongsidlength']).count().reset_index()[['strongsidlength','greek']])
a['strongs_id'] = a.strongs_id.apply(lambda x: x.split('+')[0] if len(x) > 6 else x)
a.sort_values('strongsidlength',ascending=False)
a['strongsidlength'] = a.strongs_id.apply(lambda x: len(x))
print(a.groupby(['strongsidlength']).count().reset_index()[['strongsidlength','greek']])


   strongsidlength   greek
0                5  141771
1                6      10
2               11     248
3               17       1
   strongsidlength   greek
0                5  142020
1                6      10


In [7]:
fixlist = ['CONJ +P-1NS', 'PRT-N +PRT', 'CONJ +ADV', 'CONJ +I-NSN', 'CONJ +D-NPN', 'PRT-N +ADV', 'CONJ +COND', 'CONJ +D-APN',
'CONJ +D-ASM', 'PREP+N-ASN', 'CONJ +D-NPM', 'CONJ +P-1DS', 'CONJ +D-NSM', 'A-NPM-NUI +A-NPM-NUI', 'A-APN-NUI +A-APN-NUI',
'CONJ +PRT +PRT', 'CONJ +P-1AS', 'ADV +V-AAP-NSM', 'CONJ +PRT', 'CONJ +D-APM', 'COND +CONJ', 'PREP +ADV', 'PRT +ADV', 'ADV +N-NPF']

a['morphology_id'] = a.morphology_id.apply(
    lambda x: 'CONJ' if x in fixlist and x[:4]=='CONJ' else
              'CONJ' if x in fixlist and x[:4]=='COND' else
              'PRT' if x in fixlist and x[:3]=='PRT' else
              'PREP' if x in fixlist and x[:4]=='PREP' else
              'A-NPM-NUI' if x in fixlist and x[:9]=='A-NPM-NUI' else
              'A-APN-NUI' if x in fixlist and x[:9]=='A-APN-NUI' else
              'ADV' if x in fixlist and x[:3]=='ADV' else x
)

In [8]:
# join tables
df = a[a.nestleAland==True][tagnt_fields]\
    .reset_index(drop=True)\
    .rename(columns={'strongs_id':'strongs', 'morphology_id':'morphology'})\
    .merge(b[strongs_fields], how='left', on='strongs')\
    .merge(c[tegmc_fields], how='left', on='morphology')\
    .merge(d[bcvIndex_fields], how='left', on='bcv').fillna('')


In [9]:
# renaming these to xxxxxx2 because i'm going to update each field subsequently
df = df.rename(columns={
    'function': 'function2',
    'tense': 'tense2',
    'voice': 'voice2',
    'mood': 'mood2',
    'person': 'person2',
    'case': 'case2',
    'gender': 'gender2',
    'number': 'number2',
})

## Add some ID columns

In [10]:
# build some id columns
bcv = list(df.bcv)
bcvw = ['01010101']
i = 1
bcv_ = '010101'
for item in bcv[1:]:
    if item == bcv_:
        i += 1
    else:
        i = 1
    w = ('0' + str(i))[-2:]
    bcvw.append(item+w)
    bcv_ = item

df['bcvw'] = bcvw
df['bk'] = df.bcvw.apply(lambda x: x[:2])
df['ch'] = df.bcvw.apply(lambda x: x[2:4])
df['vs'] = df.bcvw.apply(lambda x: x[4:6])
df['wd'] = df.bcvw.apply(lambda x: x[6:8])
df['bc'] = df['bk'] + df['ch']

id_columns = ['book','bcvw','bcv','bc','bk','ch','vs','wd']
df = df[id_columns + [col for col in list(df.columns) if col not in id_columns]]

## Join word frequency

In [11]:
# get the word freq for each strongs number
strongs_count = query(f"""
SELECT 
    strongs,
    lexicon,
    gloss,
    transliteration,
    count(*) as lexicon_freq_nt 
FROM df
GROUP BY 1,2,3,4
ORDER BY 1,2,3,4
""").to_df().sort_values(['lexicon_freq_nt','strongs'],ascending=[False,True]).reset_index(drop=True).reset_index().rename(columns={'index':'lexicon_id_int'})

In [12]:
strongs_count

,lexicon_id_int,strongs,lexicon,gloss,transliteration,lexicon_freq_nt
0,0,G3588,ὁ,the/this/who,ho,19856
1,1,G2532,καί,and,kai,9161
2,2,G0846,αὐτός,he/she/it/self,autos,5592
3,3,G4771,σύ,you,su,2907
4,4,G1161,δέ,then,de,2790
...,...,...,...,...,...,...
5385,5385,G9577,ὑπόλειμμα,a vestige,hupoleimma,1
5386,5386,G9992,τυπικῶς,figuratively,tupikōs,1
5387,5387,G9994,οἰκουργός,home-working,oikourgos,1
5388,5388,G9995,ἱερόθυτος,sacrificed to idols,eirothuros,1


In [13]:
df = df.merge(strongs_count[['strongs', 'lexicon_freq_nt', 'lexicon_id_int']], how='left', on='strongs')

In [14]:
df['lexicon_id'] = df.lexicon_id_int.apply(lambda x: ('000'+str(x))[-4:] )

## Redefine/Collapse/Simplify some variables 

In [15]:
df['function'] = df.function2.apply(lambda x:
    'ADJ' if x in ['Adjective',] else
    'ADV' if x in ['Adverb', 'Adverb or adverb and particle combined',] else
    'ART' if x in ['Definite article',] else
    'CONJ' if x in ['Conjunction'] else
    'PRON' if x in [
        'Correlative or Interrogative pronoun',
        'Correlative pronoun',
        'Demonstrative pronoun',
        'Indefinite pronoun',
        'Interrogative pronoun',
        'Personal pronoun',
        'Possessive pronoun',
        'Reciprocal pronoun',
        'Reflexive pronoun',
        'Relative pronoun',
    ] else
    'PRT' if x in ['Interrogative Particle', 'Negative Particle', 'Particle or Disjunctive','Interjection',] else
    'PREP' if x in ['Preposition',] else
    'NOUN' if x in ['Noun',] else
    'V' if x in ['Verb',] 
    else 'OTHER'    
)
df.groupby(['function','function2']).count()[['wd']]


wd
function function2                                    
ADJ      Adjective                                8782
ADV      Adverb                                   2500
         Adverb or adverb and particle combined     75
ART      Definite article                        19856
CONJ     Conjunction                             19238
NOUN     Noun                                    28422
PREP     Preposition                             10920
PRON     Correlative or Interrogative pronoun       27
         Correlative pronoun                       124
         Demonstrative pronoun                    1721
         Indefinite pronoun                        531
         Interrogative pronoun                     606
         Personal pronoun                        10972
         Possessive pronoun                        217
         Reciprocal pronoun                        100
         Reflexive pronoun                         402
         Relative pronoun                         1553
PRT      Interjection                              422
         Interrogative Particle                    122
         Negative Particle                        2737
         Particle or Disjunctive                   549
V        Verb                                    28111

In [16]:
df.groupby(['function',]).count()

,book,bcvw,bcv,bc,bk,ch,vs,wd,greek,english,strongs,morphology,lexicon,gloss,transliteration,definitionhtml,function2,tense2,voice2,mood2,person2,case2,gender2,number2,bookLong,lexicon_freq_nt,lexicon_id_int,lexicon_id
function,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ADJ,8782,8782,8782,8782,8782,8782,8782,8782,8782,8782,8782,8782,8782,8782,8782,8782,8782,8782,8782,8782,8782,8782,8782,8782,8782,8782,8782,8782
ADV,2575,2575,2575,2575,2575,2575,2575,2575,2575,2575,2575,2575,2575,2575,2575,2575,2575,2575,2575,2575,2575,2575,2575,2575,2575,2575,2575,2575
ART,19856,19856,19856,19856,19856,19856,19856,19856,19856,19856,19856,19856,19856,19856,19856,19856,19856,19856,19856,19856,19856,19856,19856,19856,19856,19856,19856,19856
CONJ,19238,19238,19238,19238,19238,19238,19238,19238,19238,19238,19238,19238,19238,19238,19238,19238,19238,19238,19238,19238,19238,19238,19238,19238,19238,19238,19238,19238
NOUN,28422,28422,28422,28422,28422,28422,28422,28422,28422,28422,28422,28422,28422,28422,28422,28422,28422,28422,28422,28422,28422,28422,28422,28422,28422,28422,28422,28422
PREP,10920,10920,10920,10920,10920,10920,10920,10920,10920,10920,10920,10920,10920,10920,10920,10920,10920,10920,10920,10920,10920,10920,10920,10920,10920,10920,10920,10920
PRON,16253,16253,16253,16253,16253,16253,16253,16253,16253,16253,16253,16253,16253,16253,16253,16253,16253,16253,16253,16253,16253,16253,16253,16253,16253,16253,16253,16253
PRT,3830,3830,3830,3830,3830,3830,3830,3830,3830,3830,3830,3830,3830,3830,3830,3830,3830,3830,3830,3830,3830,3830,3830,3830,3830,3830,3830,3830
V,28111,28111,28111,28111,28111,28111,28111,28111,28111,28111,28111,28111,28111,28111,28111,28111,28111,28111,28111,28111,28111,28111,28111,28111,28111,28111,28111,28111


In [17]:
## at one point, was attempting to make consistent functions by lexicon_id, but not sure that's desired on frontend

# a = (df.groupby(['lexicon_id','function']).count().reset_index()[['lexicon_id','function', 'wd']]).groupby('lexicon_id').count().reset_index()
# dups = list(a[a.wd>0].lexicon_id)
# dups
# a = df[df.lexicon_id.isin(dups)].groupby(['lexicon_id','function']).count().reset_index()[['lexicon_id', 'function', 'wd']]
# b = a.groupby('lexicon_id').max().reset_index()[['lexicon_id','wd']]
# b['flag'] = 1
# c = a.merge(b,how='left', on=['lexicon_id', 'wd'])
# d = c[c.flag==1].drop(columns=['flag','wd',]).rename(columns={'function':'function_new'})

# temp = df.merge(d, how='left', on='lexicon_id')
# e = temp[(temp.function != temp.function_new)].sort_values('lexicon_id', ascending=False)
# e.groupby(['lexicon_id','function_new']).count()

# temp[temp.lexicon_id.isin(['1254', '3195', '3485'])]
# temp['function_final'] = temp.apply(lambda x: 'ADV' if x.strongs=='G1622' else 'ADV' if x.strongs=='G2115' else '')


In [18]:
df['case'] = df.case2.apply(lambda x:
'N' if x == 'Nominative' else
'G' if x == 'Genitive' else
'D' if x == 'Dative' else
'A' if x == 'Accusative' else
'V' if x == 'Vocative' else x
)
df.groupby(['case','case2']).count()[['wd']]

,,wd
case,case2,
,,58015
A,Accusative,23319
D,Dative,12169
G,Genitive,19635
N,Nominative,23969
V,Vocative,880


In [19]:
df['tense'] = df.tense2.apply(lambda x:
'P' if x in ['Present',] else
'F' if x in ['Future', '2nd Future',] else
'I' if x in ['Imperfect',] else
'R' if x in ['Perfect', '2nd Perfect', 'Pluperfect', '2nd Pluperfect',] else
'A' if x in ['Aorist',] else
'2A' if x in ['2nd Aorist',]
else x)
df.groupby(['tense','tense2']).count()[['wd']]

wd
tense tense2                
                      109876
2A    2nd Aorist        4983
A     Aorist            6612
F     2nd Future          36
      Future            1590
I     Imperfect         1681
P     Present          11551
R     2nd Perfect        163
      2nd Pluperfect      25
      Perfect           1409
      Pluperfect          61

In [20]:
df['gender'] = df.gender2.apply(lambda x: 'F' if x=='Feminine' else 'M' if x=='Masculine' else 'N' if x=='Neuter' else x)
df.groupby(['gender', 'gender2']).count()[['wd']]

,,wd
gender,gender2,
,,63395
F,Feminine,18891
M,Masculine,41661
N,Neuter,14040


In [21]:
df['person'] = df.person2.apply(lambda x: 
'1' if x in ['1st', '1st Plural', '1st Singular',] else 
'2' if x in ['2nd', '2nd Singular'] else
'3' if x in ['3rd']
else x)
df.groupby(['person', 'person2']).count()[['wd']]

wd
person person2             
                     112826
1      1st             5470
       1st Plural         6
       1st Singular     186
2      2nd             6483
       2nd Singular      25
3      3rd            12991

In [22]:
df['voice'] = df.voice2.apply(lambda x:
'A' if x in ['Active'] else
'M' if x in ['Middle', 'Middle Deponent',] else
'P' if x in ['Passive', 'Middle or Passive', 'Passive Deponent', 'Middle or Passive Deponent',]
else x
)
df.groupby(['voice', 'voice2']).count()[['wd']]

wd
voice voice2                            
                                  109876
A     Active                       20697
M     Middle                         737
      Middle Deponent               1524
P     Middle or Passive               45
      Middle or Passive Deponent    1675
      Passive                       3110
      Passive Deponent               323

In [23]:
df['mood'] = df.mood2.apply(lambda x:
'I' if x in ['Indicative','Optative',] else
'M' if x in ['Imperative',] else
'S' if x in ['Subjunctive',]
else x
)
df.groupby(['mood', 'mood2']).count()[['wd']]

wd
mood mood2              
                  118825
I    Indicative    15617
     Optative         68
M    Imperative     1620
S    Subjunctive    1857

In [24]:
df['number'] = df.number2.apply(lambda x: 'S' if x == 'Singular' else 'P' if x == 'Plural' else x)
df.groupby(['number', 'number2']).count()[['wd']]

,,wd
number,number2,
,,38853
P,Plural,29592
S,Singular,69542


In [25]:
df = df[['bookLong', 'book', 'bcvw', 'bcv', 'bc', 'bk', 'ch', 'vs', 'wd',
       'greek', 'english', 'strongs', 'lexicon_id', 'lexicon_id_int', 'morphology', 'lexicon', 'gloss', 'transliteration',
       'function', 'tense', 'voice', 'mood', 'person', 'case', 'gender', 'number', 'lexicon_freq_nt', 'definitionhtml',]]

## Understand parsing breakouts, and Add a new Parsing field

In [26]:
print(df.groupby('function').count()[['wd']])
df.head()

             wd
function       
ADJ        8782
ADV        2575
ART       19856
CONJ      19238
NOUN      28422
PREP      10920
PRON      16253
PRT        3830
V         28111


,bookLong,book,bcvw,bcv,bc,bk,ch,vs,wd,greek,english,strongs,lexicon_id,lexicon_id_int,morphology,lexicon,gloss,transliteration,function,tense,voice,mood,person,case,gender,number,lexicon_freq_nt,definitionhtml
0,Matthew,Mat,01010101,010101,0101,01,01,01,01,Βίβλος,[The] book,G0976,1059,1059,N-NSF,βίβλος,book,biblos,NOUN,,,,,N,F,S,10,"<b>βίβλος</b>, -ου, ἡ (variant form of βύβλος..."
1,Matthew,Mat,01010102,010101,0101,01,01,01,02,γενέσεως,of [the] genealogy,G1078,1672,1672,N-GSF,γένεσις,origin,genesis,NOUN,,,,,G,F,S,5,"<b>γένεσις</b>, -εως, ἡ <BR /> (&lt; γίγνομαι..."
2,Matthew,Mat,01010103,010101,0101,01,01,01,03,Ἰησοῦ,of Jesus,G2424,0019,19,N-GSM-P,Ἰησοῦς,Jesus/Joshua,Iēsous,NOUN,,,,,G,M,S,916,"<b>Ἰησοῦς</b>, -οῦ, dative, voc. -οῦ, accusat..."
3,Matthew,Mat,01010104,010101,0101,01,01,01,04,Χριστοῦ,"Christ,",G5547,0036,36,N-GSM-T,Χριστός,Christ,Christos,NOUN,,,,,G,M,S,529,"<b>χριστός (Χρ-)</b>, -ή, -όν <BR /> (&lt; χρ..."
4,Matthew,Mat,01010105,010101,0101,01,01,01,05,υἱοῦ,son,G5207,0047,47,N-GSM,υἱός,son,uhios,NOUN,,,,,G,M,S,377,"<b>υἱός</b>, -οῦ, ὁ, [in LXX very freq. and n..."


In [27]:
df[df.function.isin(['ADV', 'PREP', 'CONJ', 'PRT', 'OTHER'])].groupby(['function','tense','voice','mood','person','case','gender','number']).count()['wd'].reset_index()

,function,tense,voice,mood,person,case,gender,number,wd
0,ADV,,,,,,,,2575
1,CONJ,,,,,,,,19238
2,PREP,,,,,,,,10920
3,PRT,,,,,,,,3830


In [28]:
df[df.function.isin(['ART', 'ADJ', 'NOUN'])].groupby(['function','tense','voice','mood','person','case','gender','number']).count()['wd'].reset_index()

,function,tense,voice,mood,person,case,gender,number,wd
0,ADJ,,,,,A,F,P,188
1,ADJ,,,,,A,F,S,453
2,ADJ,,,,,A,M,P,562
3,ADJ,,,,,A,M,S,475
4,ADJ,,,,,A,N,P,500
5,ADJ,,,,,A,N,S,745
6,ADJ,,,,,D,F,P,74
7,ADJ,,,,,D,F,S,263
8,ADJ,,,,,D,M,P,268
9,ADJ,,,,,D,M,S,213


In [29]:
df[df.function=='PRON'].groupby(['function','tense','voice','mood','person','case','gender','number']).count()['wd'].reset_index()

,function,tense,voice,mood,person,case,gender,number,wd
0,PRON,,,,,A,F,P,34
1,PRON,,,,,A,F,S,325
2,PRON,,,,,A,M,P,548
3,PRON,,,,,A,M,S,1271
4,PRON,,,,,A,N,P,419
5,PRON,,,,,A,N,S,883
6,PRON,,,,,D,F,P,63
7,PRON,,,,,D,F,S,231
8,PRON,,,,,D,M,P,600
9,PRON,,,,,D,M,S,991


In [30]:
# assign mood for infinitives & participles
df['mood'] = df.apply(lambda x: 'N' if (x.function=='V') and (x.mood=='') and (x.person=='') and (x.case=='') and (x.gender=='') and (x.number=='') else x.mood, axis=1)
df['mood'] = df.apply(lambda x: 'P' if (x.function=='V') and (x.case != "") else x.mood, axis=1)

In [31]:
df[df.function=='V'].groupby(['function','tense','voice','mood','person','case','gender','number']).count()['wd'].reset_index()

,function,tense,voice,mood,person,case,gender,number,wd
0,V,2A,A,I,1,,,P,50
1,V,2A,A,I,1,,,S,203
2,V,2A,A,I,2,,,P,67
3,V,2A,A,I,2,,,S,37
4,V,2A,A,I,3,,,P,521
5,V,2A,A,I,3,,,S,1308
6,V,2A,A,M,2,,,P,91
7,V,2A,A,M,2,,,S,134
8,V,2A,A,M,3,,,P,2
9,V,2A,A,M,3,,,S,18


In [32]:
## Parsing
df['parsing'] = df.apply(lambda x:
    (x.function + '-' + x.case + x.gender + x.number) if x.function in ['ART', 'ADJ', 'NOUN'] else
    (x.function + '-' + x.case + x.gender + x.number) if ((x.function=='PRON') & (x.person not in ['1', '2', '3'])) else
    (x.function + '-' + x.person + x.number + '-' + x.case + x.gender) if x.function=='PRON' else
    (x.function + '-' + x.tense + x.voice + x.mood) if x.mood == 'N' else
    (x.function + '-' + x.tense + x.voice + x.mood + '-' + x.case + x.gender + x.number) if x.mood == 'P' else
    (x.function + '-' + x.tense + x.voice + x.mood + '-' + x.person + x.number) if x.function == 'V'    
    else x.function
,axis=1)

In [33]:
parsing = df.groupby(['parsing']).count()['wd']

In [34]:
df = df[['bcv', 'bookLong', 'book', 'bk', 'ch', 'vs', 'wd', 'greek', 'english', 'parsing', 'lexicon', 'gloss', 'transliteration','lexicon_freq_nt', 'strongs', 'definitionhtml','lexicon_id', 'lexicon_id_int', 'function', 'tense', 'voice', 'mood', 'person', 'case', 'gender', 'number']]

## Split word and punctuation for Greek and English

In [35]:
# note: this function can return all characters for a language, 
# but using it right now to get unique characters that end
# a word, so I can separate out the puncuation characters
def get_unique_characters(var):
    word_string=''
    for word in df[var].to_list():
        word_string+=word[-1]
    unique_letters = []
    for letter in word_string:
        if letter not in unique_letters:
            unique_letters.append(letter)
    return sorted(unique_letters)

In [36]:
greek_chars = get_unique_characters('greek')
print(greek_chars[:])
english_chars = get_unique_characters('english')
print(english_chars[:])

[',', '.', ']', '¶', 'ͅ', ';', '·', 'α', 'β', 'γ', 'δ', 'ε', 'ζ', 'η', 'θ', 'ι', 'κ', 'λ', 'μ', 'ν', 'ξ', 'ο', 'ρ', 'ς', 'τ', 'υ', 'φ', 'χ', 'ψ', 'ω', 'ϊ', 'ἃ', 'ἅ', 'ἡ', 'ἢ', 'ἣ', 'ἤ', 'ἥ', 'Ἡ', 'Ἢ', 'ἰ', 'ἱ', 'ἳ', 'ἴ', 'ἵ', 'ἶ', 'ὁ', 'ὃ', 'ὅ', 'Ὁ', 'Ὃ', 'ὐ', 'ὒ', 'ὖ', 'ὗ', 'ὦ', 'Ὦ', 'ὰ', 'ά', 'ὲ', 'έ', 'ὴ', 'ή', 'ὶ', 'ί', 'ὸ', 'ό', 'ὺ', 'ύ', 'ὼ', 'ώ', 'ᾖ', 'ᾗ', 'ᾧ', 'ᾯ', 'ᾳ', 'ᾶ', 'ᾷ', '᾽', 'ῃ', 'ῆ', 'ῒ', 'ΐ', 'ῖ', 'ῦ', 'ῳ', 'ῶ', 'ῷ', '—']
[' ', '!', '"', ')', ',', '.', ':', ';', '>', '?', 'I', 'O', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'w', 'x', 'y', 'z', '—', '’', '”']


In [37]:
# determined these lists from the output above, will use in next step
greek_puncs = [',', '.', '¶', 'ͅ', ';', '·',]
english_puncs = [' ', '!', '"', ',', '.', ':', ';', '?',]

In [38]:
greek_without_puncuation = []
greek_puncuation = []
for word in list(df['greek']):    
    if len(word)>=3 and word[-3] in greek_puncs and word[-2] in greek_puncs and word[-1] in greek_puncs:            
        greek_puncuation.append(word[-3:])
        greek_without_puncuation.append(word[:-3])
    elif len(word)>=2 and word[-2] in greek_puncs and word[-1] in greek_puncs:            
        greek_puncuation.append(word[-2:])
        greek_without_puncuation.append(word[:-2])
    elif word[-1] in greek_puncs:            
        greek_puncuation.append(word[-1:])
        greek_without_puncuation.append(word[:-1])        
    else:
        greek_puncuation.append('')
        greek_without_puncuation.append(word)

df['greek2'] = greek_without_puncuation
df['greek_punc'] = greek_puncuation

In [39]:
english_without_puncuation = []
english_puncuation = []
for word in list(df['english']):    
    if len(word)>=3 and word[-3] in english_puncs and word[-2] in english_puncs and word[-1] in english_puncs:            
        english_puncuation.append(word[-3:])
        english_without_puncuation.append(word[:-3])
    elif len(word)>=2 and word[-2] in english_puncs and word[-1] in english_puncs:            
        english_puncuation.append(word[-2:])
        english_without_puncuation.append(word[:-2])
    elif word[-1] in english_puncs:            
        english_puncuation.append(word[-1:])
        english_without_puncuation.append(word[:-1])        
    else:
        english_puncuation.append('')
        english_without_puncuation.append(word)

df['english2'] = english_without_puncuation
df['english_punc'] = english_puncuation

In [40]:
df = df.rename(columns={'greek2': 'greek', 'greek':'greek2', 'english2':'english', 'english':'english2'})

In [41]:
verses = list(df['bcv'].unique())
verse_dict = {v:i for i,v in enumerate(verses,1)}
df['vs_id'] = df.bcv.apply(lambda x: verse_dict[x])

In [42]:
df = df[[
 'vs_id',
 'bcv',
 'bookLong',
 'book',
 'bk',
 'ch',
 'vs',
 'wd',
 'greek',
 'english',
 'parsing',
 'lexicon',
 'lexicon_freq_nt',
 'strongs',
 'definitionhtml',
 'lexicon_id',
 'lexicon_id_int',
 'gloss',
 'transliteration',
 'function',
 'tense',
 'voice',
 'mood',
 'person',
 'case',
 'gender',
 'number', 
 'greek_punc',
 'english_punc',]]

## Get Paradigms

In [43]:
# grouping by parsing fields
# because of various accents, keeping max count when duplicates occur
# note: lexicon_id_int parameter is ordering of strongs ids by word frequency, so 0 is the article bc it's most common
def get_paradigms(lexicon_id_int):  
    fields = ['lexicon_id', 'lexicon_id_int', 'parsing',]
    w = df[df.lexicon_id_int==lexicon_id_int]
    w['greek'] = w.greek.apply(lambda x: x.lower())
    x = w.groupby(fields+['greek']).count().reset_index()[fields+['greek', 'wd']].reset_index()
    x['wd2'] = x['wd'] + x['index']/10000
    y = x.groupby(fields).max().reset_index()[fields+['wd2']]
    y2 = x.groupby(fields).sum().reset_index()[fields+['wd']]
    y['flag'] = 1
    z = x.merge(y, how='left', on=fields+['wd2']).drop(columns=['wd','wd2',])
    z = z[z.flag==1].drop(columns=['flag',])
    z = z.merge(y2, how='left', on=fields).drop(columns=['index', 'lexicon_id_int']).rename(columns={'wd':'paradigm_freq_nt'})

    # run this to see what decisions are made in this function --> z,d = get_paradigms(18)
    # print(w.shape[0])
    # print(z.wd.sum())
    
    return z


In [44]:
# z = get_paradigms(18)

In [45]:
# step takes about 45sec
lexicon_ids_int = sorted(list(df['lexicon_id_int'].unique()))
dfs = []
for lexicon_id_int in lexicon_ids_int:
    dfs.append(get_paradigms(lexicon_id_int))
pdgm = pd.concat(dfs).reset_index(drop=True)


In [46]:
# make sure that worked...
# the difference should be exactly zero, but note decisions made for duplicate parsing records in fxn above
fields = ['lexicon_id', 'parsing',]
rows_in_gnt = df.shape[0]
paradigm_word_count = pdgm.groupby(fields).sum().paradigm_freq_nt.sum()
print(rows_in_gnt - paradigm_word_count)

0


## Create models for database

In [47]:
# paradigm model
# pdgm['pdgm_id'] = pdgm.lexicon_id + '_' + pdgm.parsing
pdgm = pdgm.rename(columns={'lexicon_id': 'pdgm_lexn_id', 'parsing':'pdgm_pars_id', 'greek':'pdgm_greek', 'paradigm_freq_nt':'pdgm_freq_nt'})
pdgm['pdgm_id'] = pdgm.pdgm_lexn_id + "_" + pdgm.pdgm_pars_id
pdgm = pdgm[['pdgm_id', 'pdgm_greek', 'pdgm_freq_nt', 'pdgm_lexn_id', 'pdgm_pars_id',]]
pdgm.head()

,pdgm_id,pdgm_greek,pdgm_freq_nt,pdgm_lexn_id,pdgm_pars_id
0,0000_ART-AFP,τὰς,340,0000,ART-AFP
1,0000_ART-AFS,τὴν,1528,0000,ART-AFS
2,0000_ART-AMP,τοὺς,730,0000,ART-AMP
3,0000_ART-AMS,τὸν,1580,0000,ART-AMS
4,0000_ART-ANP,τὰ,615,0000,ART-ANP


In [48]:
# parsing model
parsing_fields = ['parsing', 'function', 'tense', 'voice','mood', 'person', 'case', 'gender', 'number']
pars = df.groupby(parsing_fields).count()['wd'].reset_index().sort_values('parsing').rename(columns={'wd':'parsing_freq_nt'})
pars = pars.rename(columns={'parsing':'pars_id', 'function':'pars_function', 'tense':'pars_tense', 'voice':'pars_voice', 'mood':'pars_mood', 'person':'pars_person', 'case':'pars_case', 'gender':'pars_gender', 'number':'pars_number', 'parsing_freq_nt':'pars_freq_nt'})

# sorting for frontend to order paradigms easier
cols = list(pars.columns)
rank_function = {'NOUN':1, 'PRON':2, 'ADJ':3, 'V':4, 'ART':5, 'ADV':6, 'CONJ':7, 'PREP':8, 'PRT':9}
rank_tense = {'P':1, 'I':2, 'F':3, 'A':4, '2A':5, 'R':6, '':7}
rank_voice = {'A':1, 'M':2, 'P':3, '':4}
rank_mood = {'I':1, 'M':2, 'N':3, 'S':4, 'P':5, '':6}
rank_person = {'1':1, '2':2, '3':3, '':4}
rank_case = {'N':1, 'G':2, 'D':3, 'A':4, 'V':5, '':6}
rank_gender = {'M':1, 'F':2, 'N':3, '':4}
rank_number = {'S':1, 'P':2, '':3}

pars['rank_function'] = pars.pars_function.apply(lambda x: rank_function[x])
pars['rank_tense'] = pars.pars_tense.apply(lambda x: rank_tense[x])
pars['rank_voice'] = pars.pars_voice.apply(lambda x: rank_voice[x])
pars['rank_mood'] = pars.pars_mood.apply(lambda x: rank_mood[x])
pars['rank_person'] = pars.pars_person.apply(lambda x: rank_person[x])
pars['rank_case'] = pars.pars_case.apply(lambda x: rank_case[x])
pars['rank_gender'] = pars.pars_gender.apply(lambda x: rank_gender[x])
pars['rank_number'] = pars.pars_number.apply(lambda x: rank_number[x])

pars = pars.sort_values(by=[
    'rank_function',
    'rank_mood',
    'rank_tense',
    'rank_voice', 
    'rank_number',   
    'rank_person',    
    'rank_gender',
    'rank_case'
])

pars = pars.drop(columns=['rank_function', 'rank_tense', 'rank_voice', 'rank_mood', 'rank_person', 'rank_case',
'rank_gender', 'rank_number']).reset_index(drop=True).reset_index().rename(columns={
    'index':'pars_rank'
})

pars = pars[cols + ['pars_rank']]

pars.head()





,pars_id,pars_function,pars_tense,pars_voice,pars_mood,pars_person,pars_case,pars_gender,pars_number,pars_freq_nt,pars_rank
0,NOUN-NMS,NOUN,,,,,N,M,S,3590,0
1,NOUN-GMS,NOUN,,,,,G,M,S,3216,1
2,NOUN-DMS,NOUN,,,,,D,M,S,1264,2
3,NOUN-AMS,NOUN,,,,,A,M,S,2543,3
4,NOUN-VMS,NOUN,,,,,V,M,S,349,4


In [49]:
# lexicon model
lexicon_fields = ['lexicon_id', 'lexicon', 'gloss', 'transliteration', 'lexicon_freq_nt','strongs', 'definitionhtml',]
lexn = df.groupby(lexicon_fields).count()['wd'].reset_index().sort_values('lexicon_id').drop(columns=['wd',])

lexn = lexn.rename(columns={'lexicon_id':'lexn_id', 'lexicon':'lexn_greek', 'gloss':'lexn_gloss', 
'transliteration':'lexn_transliteration', 'lexicon_freq_nt':'lexn_freq_nt', 'strongs':'lexn_strongs', 'definitionhtml':'lexn_dictionaryhtml'})

lexn['lexn_strongs'] = lexn.lexn_strongs.apply(lambda x: x[1:])
temp = pdgm[['pdgm_lexn_id', 'pdgm_pars_id']].rename(columns={'pdgm_lexn_id':'lexn_id'})
temp['lexn_function'] = temp.pdgm_pars_id.apply(lambda x: x.split('-')[0])
temp2 = temp.groupby(['lexn_id',]).max().reset_index()[['lexn_id', 'lexn_function']]

lexn = lexn.merge(temp2, how='left', on='lexn_id')

# join lexnalt data from biblehub
temp = pd.read_pickle('../../pickles/lexnalt.pkl')
# temp['lexn_strongs'] = temp.strongs.apply(lambda x: ('000'+str(x))[-4:])

lexn = lexn.merge(temp, how='left', on='lexn_strongs')

lexn['greek'] = lexn.greek.fillna('')
lexn['definition'] = lexn.definition.fillna('')
lexn['usage'] = lexn.usage.fillna('')
lexn['lexn_greek_long'] = lexn.apply(lambda x: x.greek if x.greek != '' else x.lexn_greek, axis=1)
lexn['lexn_definition'] = lexn.apply(lambda x: x.definition if x.definition != '' else x.lexn_gloss, axis=1)
lexn['lexn_usage'] = lexn.apply(lambda x: x.usage if x.usage != '' else x.lexn_gloss, axis=1)
lexn['lexn_definition'] = lexn.lexn_definition.apply(lambda x: x.split('NAS Exhaustive')[0].split("Thayer's Greek")[0])
lexn['lexn_usage'] = lexn.lexn_usage.apply(lambda x: x.split('NAS Exhaustive')[0].split("Thayer's Greek")[0][:-1])
lexn = lexn[[
    'lexn_id',
    'lexn_greek',
    'lexn_greek_long',
    'lexn_transliteration',
    'lexn_gloss',
    'lexn_definition',    
    'lexn_usage',    
    'lexn_strongs',
    'lexn_function',
    'lexn_freq_nt',
]]
lexn.head(5)


,lexn_id,lexn_greek,lexn_greek_long,lexn_transliteration,lexn_gloss,lexn_definition,lexn_usage,lexn_strongs,lexn_function,lexn_freq_nt
0,0000,ὁ,"ὁ, ἡ, τό",ho,the/this/who,the,"the, the definite article",3588,ART,19856
1,0001,καί,καί,kai,and,"and, even, also","and, even, also, namely",2532,CONJ,9161
2,0002,αὐτός,"αὐτός, αὐτή, αὐτό",autos,he/she/it/self,"(1) self (emphatic) (2) he, she, it (used for ...","he, she, it, they, them, same",0846,PRON,5592
3,0003,σύ,"σύ, σοῦ, σοί, σέ",su,you,you (early mod. Eng. thou),you,4771,PRON,2907
4,0004,δέ,δέ,de,then,"but, and, now, (a connective or adversative pa...","a weak adversative particle, generally placed ...",1161,CONJ,2790


In [50]:
df['word_id'] = df.bcv + df.wd
df['word_book_id'] = df.bk
df['word_chap_id'] = df.bk + df.ch
df['word_vers_id'] = df.bcv
df.head(1)

,vs_id,bcv,bookLong,book,bk,ch,vs,wd,greek,english,parsing,lexicon,lexicon_freq_nt,strongs,definitionhtml,lexicon_id,lexicon_id_int,gloss,transliteration,function,tense,voice,mood,person,case,gender,number,greek_punc,english_punc,word_id,word_book_id,word_chap_id,word_vers_id
0,1,010101,Matthew,Mat,01,01,01,01,Βίβλος,[The] book,NOUN-NFS,βίβλος,10,G0976,"<b>βίβλος</b>, -ου, ἡ (variant form of βύβλος...",1059,1059,book,biblos,NOUN,,,,,N,F,S,,,01010101,01,0101,010101


In [51]:
# book model
book = df.groupby(['bk', 'book', 'bookLong']).count()['wd'].reset_index()
num_chapters_per_book = df.groupby(['bk']).max()['ch'].reset_index().rename(columns={'ch':'book_num_chapters'})
temp = df.groupby(['bk','ch','vs']).count()['wd'].reset_index()
num_verses_per_book = temp.groupby(['bk']).count()['wd'].reset_index().rename(columns={'wd':'book_num_verses'})
book = book\
        .merge(num_chapters_per_book, how='left', on='bk')\
        .merge(num_verses_per_book, how='left', on='bk')\
        .rename(columns={'bk':'book_id', 'bookLong':'book_name', 'book':'book_name_abbrev', 'book_num_chapters':'book_num_chapters', 'book_num_verses':'book_num_verses', 'wd':'book_num_words'})

book = book[['book_id', 'book_name', 'book_name_abbrev', 'book_num_chapters', 'book_num_verses', 'book_num_words']]   
book


,book_id,book_name,book_name_abbrev,book_num_chapters,book_num_verses,book_num_words
0,01,Matthew,Mat,28,1068,18346
1,02,Mark,Mrk,16,673,11271
2,03,Luke,Luk,24,1149,19482
3,04,John,Jhn,21,878,15635
4,05,Acts,Act,28,1002,18450
5,06,Romans,Rom,16,432,7111
6,07,1 Corinthians,1Co,16,437,6829
7,08,2 Corinthians,2Co,13,256,4477
8,09,Galatians,Gal,06,149,2230
9,10,Ephesians,Eph,06,155,2422


In [52]:
# word model
word = df[['word_id', 'greek', 'english', 'greek_punc', 'english_punc', 'parsing', 'lexicon_id', 'word_book_id', 'word_chap_id', 'word_vers_id', ]]
word = word.rename(columns={'greek':'word_greek', 'english':'word_english', 'greek_punc':'word_greek_punc', 'english_punc':'word_english_punc', 'parsing':'word_pars_id', 'lexicon_id':'word_lexn_id', })
word['word_chap_num'] = word.word_chap_id.apply(lambda x: int(x[2:]))
word['word_vers_num'] = word.word_vers_id.apply(lambda x: int(x[4:]))
word['word_word_num'] = word.word_id.apply(lambda x: int(x[6:]))
word = word.merge(book, how='left', left_on='word_book_id', right_on='book_id')\
    .rename(columns={'book_name_abbrev':'word_book_name_abbrev',})
word["word_lexn_id_copy"] = word.word_lexn_id    
word["word_pars_id_copy"] = word.word_pars_id    
word = word[[
 'word_id',
 'word_greek',
 'word_english',
 'word_greek_punc',
 'word_english_punc',
 'word_book_name_abbrev',
 'word_chap_num',
 'word_vers_num',
 'word_word_num',
 'word_lexn_id_copy',
 'word_pars_id_copy',
 'word_pars_id',
 'word_lexn_id',
 'word_book_id',
 'word_chap_id',
 'word_vers_id', 
 
 ]]
word.sample(5)

,word_id,word_greek,word_english,word_greek_punc,word_english_punc,word_book_name_abbrev,word_chap_num,word_vers_num,word_word_num,word_lexn_id_copy,word_pars_id_copy,word_pars_id,word_lexn_id,word_book_id,word_chap_id,word_vers_id
106279,11010207,πατρὸς,Father,,,Php,1,2,7,0044,NOUN-GMS,NOUN-GMS,0044,11,1101,110102
95073,07122610,εἴτε,if,,,1Co,12,26,10,0253,CONJ,CONJ,0253,07,0712,071226
51065,04040506,Σαμαρείας,of Samaria,,,Jhn,4,5,6,1024,NOUN-GFS,NOUN-GFS,1024,04,0404,040405
2990,01062615,ἀποθήκας,barns —,",",,Mat,6,26,15,1476,NOUN-AFP,NOUN-AFP,1476,01,0106,010626
90949,07020923,ἀγαπῶσιν,loving,,,1Co,2,9,23,0115,V-PAP-DMP,V-PAP-DMP,0115,07,0702,070209


In [53]:
# chapter model
chap = df.groupby('word_chap_id').count()['wd'].reset_index().rename(columns={'wd':'chap_num_words'})
temp = df.groupby(['word_chap_id', 'vs']).count()['wd'].reset_index()
num_verses_per_chapter = temp.groupby('word_chap_id').count()['wd'].reset_index().rename(columns={'wd':'chap_num_verses'})
chap = chap.merge(num_verses_per_chapter, how='left', on='word_chap_id').rename(columns={'word_chap_id':'chap_id'})
chap['chap_num'] = chap.chap_id.apply(lambda x: int(x[2:]))
chap['chap_book_id'] = chap.chap_id.apply(lambda x: x[:2])

chap.head()

,chap_id,chap_num_words,chap_num_verses,chap_num,chap_book_id
0,0101,436,25,1,01
1,0102,457,23,2,01
2,0103,335,17,3,01
3,0104,429,25,4,01
4,0105,822,48,5,01


In [54]:
# verse model
vers = df.groupby('word_vers_id').count()['wd'].reset_index().rename(columns={'word_vers_id':'vers_id', 'wd':'vers_num_words'})
print(vers.vers_num_words.sum())
vers['vers_chap_num'] = vers.vers_id.apply(lambda x: int(x[2:4]))
vers['vers_num'] = vers.vers_id.apply(lambda x: int(x[4:]))
vers['vers_book_id'] = vers.vers_id.apply(lambda x: x[:2])
vers['vers_chap_id'] = vers.vers_id.apply(lambda x: x[:4])
vers = vers.merge(book, how='left', left_on='vers_book_id', right_on='book_id')
vers['vers_ref_abbrev'] = vers.apply(lambda x: x.book_name_abbrev + '.' + " " + str(x.vers_chap_num) + ":" + str(x.vers_num), axis=1)
vers['vers_ref'] = vers.apply(lambda x: x.book_name + " " + str(x.vers_chap_num) + ":" + str(x.vers_num), axis=1)
vers['vers_chap_url'] = vers.apply(lambda x: x.book_name_abbrev + "-" + str(x.vers_chap_num) + "#verse" + str(x.vers_num), axis=1)
vers = vers.rename(columns={'book_name_abbrev':'vers_book_name_abbrev'})
vers = vers[[
    'vers_id',
    'vers_ref',
    'vers_ref_abbrev',
    'vers_chap_url',
    'vers_book_name_abbrev',
    'vers_chap_num',
    'vers_num',
    'vers_num_words',
    'vers_book_id',
    'vers_chap_id',
]]
vers.sample(5)


137987


,vers_id,vers_ref,vers_ref_abbrev,vers_chap_url,vers_book_name_abbrev,vers_chap_num,vers_num,vers_num_words,vers_book_id,vers_chap_id
3084,040530,John 5:30,Jhn. 5:30,Jhn-5#verse30,Jhn,5,30,30,04,0405
4965,060810,Romans 8:10,Rom. 8:10,Rom-8#verse10,Rom,8,10,17,06,0608
7106,200103,James 1:3,Jas. 1:3,Jas-1#verse3,Jas,1,3,9,20,2001
3469,041245,John 12:45,Jhn. 12:45,Jhn-12#verse45,Jhn,12,45,8,04,0412
6238,110210,Philippians 2:10,Php. 2:10,Php-2#verse10,Php,2,10,13,11,1102


In [55]:
# get the freq of each lexn word in each chapter
temp = query(f"""
SELECT
    concat(word_lexn_id,'_',word_chap_id) as frlc_id, 
    word_lexn_id as frlc_lexn_id,
    word_book_id as frlc_book_id,
    word_chap_id as frlc_chap_id,
    count(*) as frlc_count
FROM word
GROUP BY 1,2,3,4
ORDER BY 1,2,3,4
""").to_df()

frlc = temp\
    .merge(chap, how='left', left_on='frlc_chap_id', right_on='chap_id')\
    .merge(book, how='left', left_on='frlc_book_id', right_on='book_id')
frlc = frlc.rename(columns={'book_name_abbrev': 'frlc_book_name_abbrev', 'chap_num':'frlc_chap_num'})
frlc = frlc[['frlc_id', 'frlc_book_name_abbrev', 'frlc_chap_num', 'frlc_count', 'frlc_lexn_id',]]
frlc

# get the freq of each lexn word in each book
temp = query(f"""
SELECT
    concat(word_lexn_id,'_',word_book_id) as frlb_id, 
    word_lexn_id as frlb_lexn_id,
    word_book_id as frlb_book_id,    
    count(*) as frlb_count
FROM word
GROUP BY 1,2,3
ORDER BY 1,2,3
""").to_df()

frlb = temp.merge(book, how='left', left_on='frlb_book_id', right_on='book_id')
frlb = frlb.rename(columns={'book_name_abbrev': 'frlb_book_name_abbrev', })
frlb = frlb[['frlb_id', 'frlb_book_name_abbrev', 'frlb_count', 'frlb_lexn_id',]]
frlb


,frlb_id,frlb_book_name_abbrev,frlb_count,frlb_lexn_id
0,0000_01,Mat,2788,0000
1,0000_02,Mrk,1509,0000
2,0000_03,Luk,2646,0000
3,0000_04,Jhn,2186,0000
4,0000_05,Act,2709,0000
...,...,...,...,...
18890,5385_06,Rom,1,5385
18891,5386_07,1Co,1,5386
18892,5387_17,Tit,1,5387
18893,5388_07,1Co,1,5388


In [58]:
# combine all into one table
gnt = word\
        .merge(vers, how='left', left_on="word_vers_id", right_on="vers_id")\
        .merge(chap, how='left', left_on="word_chap_id", right_on="chap_id")\
        .merge(book, how='left', left_on="word_book_id", right_on="book_id")\
        .merge(pars, how='left', left_on="word_pars_id", right_on="pars_id")\
        .merge(lexn, how='left', left_on="word_lexn_id", right_on="lexn_id")\
        

In [59]:
# save pickles for django models
word.to_pickle('../../pickles/word.pkl')
pars.to_pickle('../../pickles/pars.pkl')
lexn.to_pickle('../../pickles/lexn.pkl')
pdgm.to_pickle('../../pickles/pdgm.pkl')
book.to_pickle('../../pickles/book.pkl')
chap.to_pickle('../../pickles/chap.pkl')
vers.to_pickle('../../pickles/vers.pkl')
gnt.to_pickle('../../pickles/gnt.pkl')

In [61]:
# step takes about 2.5 minutes
with pd.ExcelWriter('../../../api/pickles/gnt.xlsx') as writer:  
    word.to_excel(writer, sheet_name='word')
    pars.to_excel(writer, sheet_name='pars')
    lexn.to_excel(writer, sheet_name='lexn')
    pdgm.to_excel(writer, sheet_name='pdgm')
    book.to_excel(writer, sheet_name='book')
    chap.to_excel(writer, sheet_name='chap')
    vers.to_excel(writer, sheet_name='vers')
    gnt.to_excel(writer, sheet_name='gnt')

